In [1]:
from fastbook import *

In [2]:
from fastai.text.all import *
path = untar_data(URLs.IMDB)

In [3]:
files = get_text_files(path, folders = ['train', 'test', 'unsup'])

In [4]:
txt = files[0].open().read(); txt[:75]

'SPOILER ALERT!!!!<br /><br />I had just watched the extended version of Bed'

# Word Tokenization

In [5]:
WordTokenizer?

In [6]:
spacy = WordTokenizer()
toks = first(spacy([txt]))
print(coll_repr(toks, 30))

(#148) ['SPOILER','ALERT!!!!<br','/><br','/>I','had','just','watched','the','extended','version','of','Bedknobs','and','Broomsticks','.','Though','I','did','like','the','extended','version',',','I','wish','they','would','have','left','the'...]


In [7]:
# coll_repr??

In [8]:
first(spacy(['The U.S. dollar $1 is $1.00.']))

(#9) ['The','U.S.','dollar','$','1','is','$','1.00','.']

In [9]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))

(#169) ['xxbos','xxup','spoiler','xxup','alert','xxrep','4','!','\n\n','i','had','just','watched','the','extended','version','of','xxmaj','bedknobs','and','xxmaj','broomsticks','.','xxmaj','though','i','did','like','the','extended','version'...]


In [10]:
replace_maj??

In [11]:
coll_repr(tkn('&copy;   Fast.ai www.fast.ai/INDEX'), 31)

"(#11) ['xxbos','©','xxmaj','fast.ai','xxrep','3','w','.fast.ai','/','xxup','index']"

# Subword Tokenization

In [12]:
txts = L(o.open().read() for o in files[:2000])

In [13]:
# def subword(sz):
#     sp = SubwordTokenizer(vocab_sz=sz)
#     sp.setup(txts)
#     return ' '.join(first(sp([txt]))[:40])

In [14]:
# subword(1000)

In [15]:
# subword(200)

In [16]:
# subword(10000)

In [17]:
# subword(200)

# Numericalization

In [18]:
toks = tkn(txt)
print(coll_repr(tkn(txt), 31))

(#169) ['xxbos','xxup','spoiler','xxup','alert','xxrep','4','!','\n\n','i','had','just','watched','the','extended','version','of','xxmaj','bedknobs','and','xxmaj','broomsticks','.','xxmaj','though','i','did','like','the','extended','version'...]


In [19]:
toks200 = txts[:200].map(tkn)
toks200[0]

(#169) ['xxbos','xxup','spoiler','xxup','alert','xxrep','4','!','\n\n','i'...]

In [20]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab,20)

"(#2192) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the',',','.','and','a','of','to','is','in','it','i'...]"

In [21]:
nums = num(toks)[:20]; nums

TensorText([   2,    7, 1265,    7,    0,    5,  499,   55,   27,   19,   95,   73,  327,    9, 1601,  259,   14,    8,    0,   12])

In [22]:
' '.join(num.vocab[o] for o in nums)

'xxbos xxup spoiler xxup xxunk xxrep 4 ! \n\n i had just watched the extended version of xxmaj xxunk and'

# Putting Our Texts into Batches for a Language Model

In [23]:
nums200 = toks200.map(num)

In [24]:
dl = LMDataLoader(nums200)

In [25]:
x,y = first(dl)
x.shape,y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

In [26]:
' '.join(num.vocab[o] for o in x[0][:20])

'xxbos xxup spoiler xxup xxunk xxrep 4 ! \n\n i had just watched the extended version of xxmaj xxunk and'

In [27]:
' '.join(num.vocab[o] for o in y[0][:20])

'xxup spoiler xxup xxunk xxrep 4 ! \n\n i had just watched the extended version of xxmaj xxunk and xxmaj'

# Training a Text Classifier


In [28]:
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=32, seq_len=80)

In [29]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj edge of xxmaj madness is a tale about a woman in the 1800 's who gets hand - picked by dirty xxmaj scotsman who ca n't keep his penis in his pants . xxmaj he just so happens to have a younger brother who is against rape and kills him , but continuously says it was an accident . xxmaj this makes his wife go crazy , she gets delusional and she takes the fall because she loves","xxmaj edge of xxmaj madness is a tale about a woman in the 1800 's who gets hand - picked by dirty xxmaj scotsman who ca n't keep his penis in his pants . xxmaj he just so happens to have a younger brother who is against rape and kills him , but continuously says it was an accident . xxmaj this makes his wife go crazy , she gets delusional and she takes the fall because she loves xxmaj"
1,"disgusted by them , though i have to say some of them look truly embarrassed to be there . \n\n xxmaj as i said the story lines are nothing but recycled predicable crap , you know the drill , boy meets girl , boy and girl flirt ad nauseam , boy kisses girl , girl fights with boy , girl breaks up with boy , rinse wash repeat , honestly the writers of this show should be blasted from a","by them , though i have to say some of them look truly embarrassed to be there . \n\n xxmaj as i said the story lines are nothing but recycled predicable crap , you know the drill , boy meets girl , boy and girl flirt ad nauseam , boy kisses girl , girl fights with boy , girl breaks up with boy , rinse wash repeat , honestly the writers of this show should be blasted from a cannon"


# Fine-Tuning the Language Model

In [30]:
# learn = language_model_learner(
#     dls_lm, AWD_LSTM, drop_mult=0.3, 
#     metrics=[accuracy, Perplexity()]).to_fp16()

In [31]:
# learn.fit_one_cycle(1, 2e-2)

# Saving and Loading Models

In [32]:
# learn.save('1epoch')

In [33]:
learn = learn.load('1epoch')

In [34]:
# learn.unfreeze()
# learn.fit_one_cycle(10, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.855602,3.767543,0.316489,43.273594,51:39
1,3.789299,3.773195,0.317775,43.518898,50:52
2,3.753580,3.746979,0.321367,42.392799,50:56
3,3.714319,3.717806,0.325202,41.173962,50:52
4,3.642033,3.686207,0.328952,39.893227,50:51
5,3.558468,3.652498,0.332970,38.570908,52:05
6,3.482125,3.621438,0.336450,37.391300,51:55
7,3.399919,3.602140,0.339275,36.676647,50:36
8,3.322546,3.594292,0.340580,36.389942,50:36
9,3.268277,3.598757,0.340170,36.552765,50:38


In [35]:
# learn.save_encoder('finetuned')

# Text Generation

In [36]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [37]:
print("\n".join(preds))

i liked this movie because of the beautiful photography and the good acting . At least , the actors seemed to be very " critic " and / or " actors " . 

 This was the kind of movie that gives the
i liked this movie because it is the second best film shown in recent years and one of the most beautiful pictures i have seen so far . Everyone should see this movie . It has a great problem in showing how society


# Creating the Classifier DataLoaders

In [52]:
# torch.cuda.empty_cache()

In [58]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=32, seq_len=72)

In [59]:
dls_clas.show_batch(max_n=3)

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,"xxbos xxmaj some have praised _ xxunk _ as a xxmaj disney adventure for adults . i do n't think so -- at least not for thinking adults . \n\n xxmaj this script suggests a beginning as a live - action movie , that struck someone as the type of crap you can not sell to adults anymore . xxmaj the "" crack staff "" of many older adventure movies has been done well before , ( think _ the xxmaj dirty xxmaj dozen _ ) but _ atlantis _ represents one of the worse films in that motif . xxmaj the characters are weak . xxmaj even the background that each member trots out seems stock and awkward at best . xxmaj an xxup md / xxmaj medicine xxmaj man , a tomboy mechanic whose father always wanted sons , if we have not at least seen these before",neg
2,"xxbos xxmaj well , on it 's credit side ( if it can be said to have one ) , xxmaj timothy xxmaj hines xxup did manage to capture the original setting of xxup h.g . xxmaj wells ' outstanding novella . xxmaj but other than that - well , to call a spade a spade - it sucks bigtime . xxmaj what the xxmaj master xxmaj ed xxmaj wood could have done with the alleged $ 20 million dollar budget ! xxmaj timothy xxmaj hines really does make xxmaj mr . xxmaj wood , who was a flawed genius anyway , look like the best filmmaker of all time . xxmaj the special effects ( i guess you 'd call them that ) are not even up to computer game standards . xxmaj the acting is , well , perhaps about dinner theater comparable , and the accents are",neg


In [60]:
nums_samp = toks200[:10].map(num)

In [61]:
nums_samp.map(len)

(#10) [169,426,268,176,142,398,323,263,242,236]

In [62]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy).to_fp16()

In [63]:
learn = learn.load_encoder('finetuned')

# Fine-Tuning the Classifier

In [64]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.235497,0.171363,0.934680,05:59


In [65]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.231893,0.154877,0.943080,06:19


In [66]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.193997,0.143832,0.947920,07:39


In [69]:
# learn.unfreeze()
# learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))